In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
import seaborn as sns

availability_data = pd.read_csv('/Users/aryamantepal/Documents/programs/Breakthrough Tech AI MIT/AI Studio/Customer_Sentiment_Analysis/Datasets/AvailabilityData.csv', sep="\t")

availability_data.columns

/Users/aryamantepal/anaconda3/envs/salesforce/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Index(['Date', 'Availability DownTime Duration in hours',
       'Count of Customers Affected', 'Regions Affected'],
      dtype='object')

In [2]:
availability_data['Date'] = pd.to_datetime(availability_data['Date'])

prophet_df = availability_data.rename(columns={
    'Date': 'ds', 
    'Availability DownTime Duration in hours': 'y'
})

model = Prophet(
    changepoint_prior_scale=0.05,  
    seasonality_prior_scale=10,  
    holidays_prior_scale=10        
)

model.add_regressor('Count of Customers Affected')
model.add_regressor('Downtime_Severity')

prophet_df['Downtime_Severity'] = availability_data['Availability DownTime Duration in hours'] / availability_data['Count of Customers Affected']

model.fit(prophet_df)

future = model.make_future_dataframe(periods=30)  
future['Count of Customers Affected'] = np.concatenate([
    availability_data['Count of Customers Affected'], 
    np.full(30, availability_data['Count of Customers Affected'].mean())
])
future['Downtime_Severity'] = np.concatenate([
    prophet_df['Downtime_Severity'], 
    np.full(30, prophet_df['Downtime_Severity'].mean())
])

forecast = model.predict(future)

06:15:37 - cmdstanpy - INFO - Chain [1] start processing
06:15:37 - cmdstanpy - INFO - Chain [1] done processing


In [3]:
forecast_components = model.plot_components(forecast)
plt.tight_layout()
plt.savefig('outreach_timing_components.png')
plt.close()

/Users/aryamantepal/anaconda3/envs/salesforce/lib/python3.10/site-packages/prophet/plot.py:228: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/Users/aryamantepal/anaconda3/envs/salesforce/lib/python3.10/site-packages/prophet/plot.py:228: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()


In [4]:
# Identify optimal outreach windows
def analyze_outreach_windows(forecast):
    forecast['forecast_lower'] = forecast['yhat_lower']
    forecast['forecast_upper'] = forecast['yhat_upper']
    
    forecast['confidence_interval'] = forecast['forecast_upper'] - forecast['forecast_lower']
    
    optimal_windows = forecast.nsmallest(5, 'confidence_interval')
    
    return optimal_windows[['ds', 'yhat', 'forecast_lower', 'forecast_upper', 'confidence_interval']]

optimal_outreach_times = analyze_outreach_windows(forecast)

def analyze_regional_impact(availability_data):
    availability_data['Regions'] = availability_data['Regions Affected'].str.split(', ')
    
    region_impact = availability_data.explode('Regions')
    
    region_summary = region_impact.groupby('Regions').agg({
        'Availability DownTime Duration in hours': ['mean', 'sum'],
        'Count of Customers Affected': ['mean', 'sum']
    }).reset_index()
    
    region_summary.columns = ['Region', 'Avg_Downtime', 'Total_Downtime', 'Avg_Customers_Affected', 'Total_Customers_Affected']
    
    return region_summary

In [8]:
print("Optimal Outreach Windows:")
print(optimal_outreach_times)


regional_impact = analyze_regional_impact(availability_data)
print("\nRegional Impact Summary:")
print(regional_impact)


optimal_outreach_times.to_csv('optimal_outreach_windows.csv', index=False)
regional_impact.to_csv('regional_impact_summary.csv', index=False)


plt.figure(figsize=(12, 6))
plt.plot(forecast['ds'], forecast['yhat'], label='Predicted Downtime')
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], alpha=0.3)
plt.title('Predicted Downtime with Confidence Interval')
plt.xlabel('Date')
plt.ylabel('Predicted Downtime Duration (hours)')
plt.legend()
plt.tight_layout()
plt.savefig('outreach_timing_forecast.png')
plt.close()

print("\nOutputs generated:")
print("1. optimal_outreach_windows.csv - Detailed optimal outreach windows")
print("2. regional_impact_summary.csv - Analysis of regional impacts")
print("3. outreach_timing_components.png - Forecast components")
print("4. outreach_timing_forecast.png - Forecast visualization")

Optimal Outreach Windows:
            ds       yhat  forecast_lower  forecast_upper  confidence_interval
70  2024-03-11  21.256374       14.470132       28.038434            13.568303
68  2024-03-09  11.089421        4.138454       17.753103            13.614649
139 2024-05-19  14.789203        7.847984       21.538783            13.690799
16  2024-01-17  10.699471        3.980390       17.728880            13.748489
187 2024-07-06  11.788602        4.946416       18.714492            13.768076

Regional Impact Summary:
      Region  Avg_Downtime  Total_Downtime  Avg_Customers_Affected  \
0  Australia     11.104167             533            24775.208333   
1      China     12.038462             626            25284.538462   
2     Europe     11.052632             630            29391.578947   
3      India     11.333333             612            25425.462963   
4      Japan     11.562500             740            22345.296875   
5        USA      9.705882             495            